In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import librosa.display as lib_display
import numpy as np

In [2]:
emotion_path_csv = os.path.abspath(os.path.join("Features","Emotion_path_mapped.csv"))
df = pd.read_csv(emotion_path_csv,index_col=False).drop(["Unnamed: 0"],axis=1)
df.head(10)

,Emotion,Path
0,neutral,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
1,neutral,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
2,neutral,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
3,neutral,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
4,calm,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
5,calm,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
6,calm,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
7,calm,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
8,calm,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...
9,calm,p:\TMSL\SEM3_Minor_Projet\Notebooks\Datasets\R...


# Main feature extraction function

In [5]:
def extract_features(data,sample_rate):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

## Data Augmentation Function

---
> Adding

> - Noise

> - Stretch

> - Shift

> - Pitch

In [9]:

def noise(data):
    noise_amp = 0.025*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    """
        data: Main Audio
        Rate: The rate in which the audio is being slowed or fast
        FROMULA/ Function  : librosa.effects.time_stretch(data, rate)
    """
    return librosa.effects.time_stretch(data, rate)

def shift(data):
    """
        data: it takes only audio data as parameter
        returns np array of new audio
    """
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)


----
##### Get Feature Function to apply random function amongst (pitch , stretch ...)

#### and extract feature of that audio
---

In [26]:
import random as rnd

In [39]:
def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    result=[]
    rand_no_list = [1,2,3,4,5]
    k  = rnd.choice(rand_no_list)
    # print(k)
    if k==1:
        # print("in 1")
        res1 = extract_features(data,sample_rate)
        result = np.array(res1)
    elif k==2:
        # print("in 2")
        noise_data = noise(data)
        res1 = extract_features(noise_data,sample_rate)
        result = np.array(res1)
    elif k==3:
        # print("in 3")
        stretch_data = stretch(data)
        res1 = extract_features(stretch_data,sample_rate)
        result = np.array(res1)
    elif k==4:
        # print("in 4")
        pitch_data = pitch(data,sampling_rate=sample_rate,pitch_factor=rnd.random())
        res1 = extract_features(pitch_data,sample_rate)
        result = np.array(res1)
    else:
        # print("in 5")
        shift_data = shift(data)
        res1 = extract_features(shift_data,sample_rate)
        result = np.array(res1)
    
    # print(f"reslt size {result.shape}")    
    return result

---
### feature extraction process for final Test Data
----

In [40]:
from tqdm.notebook import tqdm_notebook as prog_bar

In [41]:

Features,Labels = [],[]
for emo,aud_path in prog_bar(zip(df.Emotion,df.Path),total = 1440):
    aud_features = get_features(aud_path) 
    Features.append(aud_features)
    Labels.append(emo)

  0%|          | 0/1440 [00:00<?, ?it/s]

In [42]:
Df = pd.DataFrame(Features)
Df['labels'] = Labels
Df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.300184,0.738577,0.742603,0.724332,0.726936,0.683394,0.653090,0.696985,0.748464,0.751690,...,0.000004,0.000003,0.000002,0.000002,0.000005,0.000008,5.375256e-06,2.270875e-06,1.642310e-07,neutral
1,0.270354,0.683518,0.730694,0.716595,0.679522,0.675306,0.669987,0.631418,0.686722,0.713734,...,0.000007,0.000007,0.000007,0.000007,0.000012,0.000009,8.465726e-06,2.638482e-06,1.788962e-07,neutral
2,0.307369,0.716178,0.761089,0.758008,0.758388,0.734855,0.649470,0.630941,0.680735,0.738601,...,0.000013,0.000013,0.000015,0.000020,0.000028,0.000048,2.483219e-05,8.083573e-06,6.468821e-07,neutral
3,0.298019,0.768713,0.744220,0.736126,0.739507,0.687656,0.680441,0.684369,0.737645,0.710171,...,0.000020,0.000019,0.000025,0.000080,0.000040,0.000037,3.107767e-05,1.029449e-05,6.636600e-07,neutral
4,0.154713,0.622998,0.607691,0.712971,0.753272,0.674684,0.643868,0.681713,0.636211,0.655982,...,0.000002,0.000002,0.000002,0.000002,0.000005,0.000001,9.390579e-07,3.889406e-07,3.605377e-08,calm


# Saving the Dataframe as final test data

In [43]:
save_path = os.path.join('Features','Final_Test_Data.csv')
Df.to_csv(save_path, index=False)